In [1]:
import requests
import json
import pandas as pd
from langchain.document_loaders import DataFrameLoader

In [82]:
vectordb_url="http://vectordb.runai-vast.svc.cluster.local/retrieve-chunks/"

query = "The most common systems used in eSport games for matchmaking are skill and rank-based systems."

query_string = {'query':query}

response = requests.post(vectordb_url,json=query_string)

results = pd.DataFrame(response.json()['results'])

print (results)

                         element_id  \
0  4b38edcd8cb19551bf707426d01c346c   
1  e9341ada70ee72775f5305af251aa492   
2  c0e3da97370455961560e4975bd528a9   
3  144343272cd58584a994582fd993ccee   
4  8d943843d36284a126a15acb5c5bf3a7   
5  a40a69099854c04bb0c4e96d2beee944   
6  70a994cc0493ba9317b102a3f6d2ba98   
7  4736e10982d48ce915697f6bf7fce972   
8  7a8ca1c2fe1895949a7255ecfa272554   
9  8176a1390a48413a838b44b1bddea738   

                                        cleaned_text    file_directory  \
0  A large body of research on team formation onl...  /tmp/pdf_chunker   
1  based on processing of whole-brain tractograms...  /tmp/pdf_chunker   
2  5.2 Benchmarking Methods Benchmarking SM appro...  /tmp/pdf_chunker   
3  Hidden Markov Models (HMM), often factorial HM...  /tmp/pdf_chunker   
4  Both research articles conducted on industrial...  /tmp/pdf_chunker   
5  DISCUSSION We systematically reviewed studies ...  /tmp/pdf_chunker   
6  On the other hand, considering the use of mark...

In [84]:
rerank_url="http://rerankqa-mistral-4b.runai-genai.svc.cluster.local/v1/ranking"

start_string = '{"model": "nvidia/nv-rerankqa-mistral-4b-v3","query": {"text":"' + query + '"},"passages": [{"text": "'
 
middle_string = '"},{"text": "'.join(results['cleaned_text'])

middle_string = middle_string.encode("ascii","ignore")

middle_string = middle_string.decode()

#middle_string = 'format check'

end_string='"}],"truncate": "END"}'

rerank_string = start_string+middle_string+end_string

#rerank_string = '{"model": "nvidia/nv-rerankqa-mistral-4b-v3","query": {"text": "which way should i go?"},"passages": [{"text": "two roads diverged in a yellow wood, and sorry i could not travel both and be one traveler, long i stood and looked down one as far as i could to where it bent in the undergrowth;"},{"text": "then took the other, as just as fair, and having perhaps the better claim because it was grassy and wanted wear, though as for that the passing there had worn them really about the same,"},{"text": "and both that morning equally lay in leaves no step had trodden black. oh, i marked the first for another day! yet knowing how way leads on to way i doubted if i should ever come back."},{"text": "i shall be telling this with a sigh somewhere ages and ages hense: two roads diverged in a wood, and i, i took the one less traveled by, and that has made all the difference."}],"truncate": "END"}'

#print (rerank_string)
json_object = json.loads(rerank_string)

print (json.dumps(json_object, indent=1))

{
 "model": "nvidia/nv-rerankqa-mistral-4b-v3",
 "query": {
  "text": "The most common systems used in eSport games for matchmaking are skill and rank-based systems."
 },
 "passages": [
  {
   "text": "A large body of research on team formation only considers the team itself in isolation from the competitive environment. A study has involved multi-stage stochastic algorithms for team formation, but only to optimize two properties: cost and competency [11]. Others have focused on optimizing team formation while minimizing fault lines within the team, but without trying to maximize the competitiveness [12]. Others also focus on optimizing teams based on the generally accepted nine Belbin team roles or the experts and laymen problem [13], [14]. Such an approach is not generalizable in domain where flat hierarchy dominates the teams. Algorithms are developed for team formation in the field of software development, often using bio-inspired algorithms such as genetic algorithms [15], [16]. H

In [91]:
headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json'
}

response = requests.post(rerank_url, headers=headers, data=rerank_string)

ranking = pd.DataFrame(response.json()['rankings'])

citing_text = ""

if ranking['logit'].loc[0] > -10:
    citing_text = results['cleaned_text'].loc[ranking['index'].loc[0]]

print(citing_text)

print(ranking)

A large body of research on team formation only considers the team itself in isolation from the competitive environment. A study has involved multi-stage stochastic algorithms for team formation, but only to optimize two properties: cost and competency [11]. Others have focused on optimizing team formation while minimizing fault lines within the team, but without trying to maximize the competitiveness [12]. Others also focus on optimizing teams based on the generally accepted nine Belbin team roles or the experts and laymen problem [13], [14]. Such an approach is not generalizable in domain where flat hierarchy dominates the teams. Algorithms are developed for team formation in the field of software development, often using bio-inspired algorithms such as genetic algorithms [15], [16]. However, these often require fine-grained data for each member, using data mining techniques or manual intervention for the dataset creation [16]–[25]. Such precise data is not available in our domain wh